## Задание 1.


Написать алгоритм получения данных по вакансиям для IT-специалистов с одного из работных сайтов.



Алгоритм позволяет получить информацию о количестве вакансий по направлениям Data Analyst, Data Science и Data Engineer в разрезе уровней (Junior, Middle, Senior).


За основу можно взять как один регион (НН, Москва и т.д.)  так и несколько.


Результат представить в табличном и/или графическом виде.

In [34]:
import requests

url = "https://api.hh.ru/areas"

response = requests.get(url)
areas = response.json()

In [35]:
def get_areas_recursive(areas):
    all_areas = {}
    for area in areas:
        all_areas[area.get("name")] = area.get("id")
        if area.get("areas"):
            all_areas.update(get_areas_recursive(area.get("areas")))
    return all_areas

all_areas = get_areas_recursive(areas)

In [36]:
all_areas = dict(sorted(all_areas.items(), key=lambda x: x[0]))

In [37]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

area_id = 1
areas_names = list(all_areas.keys())

def load_vacancies(area_name):
    global area_id
    area_id = all_areas[area_name]
    grades = ["Junior", "Middle", "Senior"]
    work_types = ["Data Analyst", "Data Science", "Data Engineer"]
    results = {}
    
    for work_type in work_types:
        for grade in grades:
            url = f"https://api.hh.ru/vacancies?area={area_id}&text={grade} {work_type}"
            response = requests.get(url)
            data = response.json()
            if not results.get(work_type):
                results[work_type] = {}
            results[work_type][grade] = data.get("found")
    
    data_found = False
    for work_type in work_types:
        if all(results[work_type][grade] == 0 for grade in grades):
            print(f"No {work_type} data found in {area_name}")
        else:
            data_found = True
            plt.title(f"{work_type} in {area_name} by grades")
            plt.bar(results[work_type].keys(), results[work_type].values())
            plt.xticks(rotation=45, ha="right")
            plt.show()

    if not data_found:
        return
    else:
        works = []
        works_counts = []
        
        for work_type in work_types:
            for grade in grades:
                works.append(f"{grade} {work_type}")
                works_counts.append(results[work_type][grade])
    
        plt.title(f"All vacancies in {area_name} by grades")    
        plt.bar(works, works_counts)
        plt.xticks(rotation=45, ha="right")
        plt.show()

widgets.interact(load_vacancies, area_name=areas_names);

interactive(children=(Dropdown(description='area_name', options=('1-й Воин', '1-я Михайловка', '1-я Семеновка'…